# End to end tutorial for using the ne Argilla SDK

This tutorial encomposses the full workflow of using the Argilla SDK to manage human feedback tasks. The main three areas are:

1. **Setting up a project**:
   - **Connecting to an Argilla server**:
   - **Creating a workspace**:
   - **Creating users**:
2. **Creating a dataset**:
   - **Defining the dataset's feedback task**:
   - **Adding records to the dataset**:
3. **Labelling the dataset**:
   - **Assigning records to users**:
   - **Adding suggestions and responses**:

We will work through each of these areas and show three depths of control: A line, B line, and C line. The A line is the simplest and most abstracted, the B line is a bit more detailed, and the C line is the most detailed and flexible.


In [1]:
import argilla_sdk as rg

# 1. Setting up a project

- Connecting to an Argilla server
- Creating a workspace
- Creating users


In [2]:
client = rg.Argilla(api_url="http://localhost:6900", api_key="owner.apikey")

In [3]:
client.workspaces

datasets,name,id,updated_at
685,admin,47a2ace0-a9b2-4dde-9710-0656646a3de3,2024-01-31T18:48:50.856584
1,new_ws,b4b7410c-7014-4ae4-a923-ec095d4a7272,2024-03-19T05:59:06.362275
0,end-to-end,e5bf1534-b89a-4245-9a30-99284710a1a9,2024-04-08T13:41:11.967896


In [4]:
ws = rg.Workspace("end-to-end", client=client)

In [5]:
try:
    ws.create()
except Exception as e:
    print("Workspace exists")

Workspace exists


In [6]:
user = rg.User(username="end-user", password="password", client=client)

try:
    ws.create()
except Exception as e:
    print("User exists")

client.users

User exists


username,id,role,updated_at
owner,b1391ace-f089-4160-a26b-b17a966dc2b5,owner,2024-01-31T18:48:50.855527
admin,5bb2a532-36f6-4af8-b5b8-ad2fbba4b7d3,admin,2024-01-31T18:48:52.994136
argilla,d12eb6df-85fd-4902-a7cd-7d94ed3eee07,annotator,2024-01-31T18:48:55.167978
test_287,9edfc138-a359-469e-94f6-f49da5221d3c,annotator,2024-03-20T12:34:23.029706
test_285,d595cfdd-c963-4c72-a579-4e108ea50393,annotator,2024-03-20T12:35:09.385889
test_892,e266e491-7171-4d26-b1c7-03322814cc30,annotator,2024-03-20T12:35:24.789664
test_326,31783991-75bb-4bb8-a82f-e30196933698,annotator,2024-03-20T13:59:50.924672
test_144,8a8746ac-e710-400b-a36b-64938f01c751,annotator,2024-03-20T14:15:39.400628
test_483,fe958d9e-d9cb-485a-b331-99469a35d196,annotator,2024-03-20T14:16:23.643043
test_142,a219673e-adcf-430f-9451-d4bc5458c8a6,annotator,2024-03-20T19:14:21.523621


# 2. Creating a dataset

- Defining the dataset's feedback task
- Adding records to the dataset


In [7]:
!pip install -qqq datasets

from datasets import load_dataset

ds = load_dataset("imdb", split="train[:100]").to_list()
ds = [{"text": x["text"], "label": "positive" if x["label"] == 1 else "negative"} for x in ds]

print(ds[:5])

/home/ben/code/argilla-python/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far b

In [8]:
# Defining the dataset's feedback task
settings = rg.Settings(
    guidelines="Classify the articles into one of the four categories.",
    fields=[
        rg.TextField(
            name="text",
            title="Text from the article",
            use_markdown=False,
        ),
    ],
    questions=[
        rg.LabelQuestion(
            name="label",
            title="In which category does this article fit?",
            labels=["positive", "negative"],
        )
    ],
)

In [11]:
dataset = rg.Dataset(
    name="IMDB-dataset-8",
    workspace_id=str(client.workspaces[0].id),
    settings=settings,
    client=client,
)

In [13]:
dataset.publish()
dataset.records.add(records=ds)

# 3. Collecting the dataset

In [14]:
dataset.records.pull()
records = dataset.records.to_list()
records[:5]

[{'external_id': UUID('23c024eb-e7c7-4d65-b13b-c9fd30ac7661'),
  'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered po